# LSTM Example 2: Estimation

In [1]:
import time
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import minimize

def sigmoid(x): 
    return 1. / (1 + np.exp(-x))

In [2]:
def param_toWeights(param):
    # input size: M x 1
    # hidden state size: K x 1    
    # output size: N x 1
    input_weights_U_i = np.array([[param[0], param[1]],
                                [param[2], param[3]],
                                [param[4],param[5]]]) 
    hidden_weights_W_i =  np.array([[param[6], param[7], param[8]],
                                  [param[9], param[10], param[11]],
                                  [param[12], param[13], param[14]]]) 
    hidden_bias_i = np.array([[param[15]],
                             [param[16]],
                             [param[17]]]) 
    # Forget Gate
    input_weights_U_f = np.array([[param[18], param[19]],
                                [param[20], param[21]],
                                [param[22],param[23]]]) 
    hidden_weights_W_f =  np.array([[param[24], param[25], param[26]],
                                  [param[27], param[28], param[29]],
                                  [param[30], param[31], param[32]]]) 
    hidden_bias_f = np.array([[param[33]],
                             [param[34]],
                             [param[35]]]) 
    # Output gate
    input_weights_U_o = np.array([[param[36], param[37]]]) 
    hidden_weights_W_o =  np.array([[param[38], param[39], param[40]]]) 
    hidden_bias_o = np.array([[param[41]]])
    # New Memory
    input_weights_U_new = np.array([[param[42], param[43]],
                                [param[44], param[45]],
                                [param[46],param[47]]]) 
    hidden_weights_W_new =  np.array([[param[48], param[49], param[50]],
                                  [param[51], param[52], param[53]],
                                  [param[54], param[55], param[56]]]) 
    return input_weights_U_i, hidden_weights_W_i, hidden_bias_i,\
    input_weights_U_f, hidden_weights_W_f, hidden_bias_f,\
    input_weights_U_o, hidden_weights_W_o, hidden_bias_o,\
    input_weights_U_new, hidden_weights_W_new

def LSTM_Loss(param,*args):
    #Initialize weights and biases
    input_weights_U_i, hidden_weights_W_i, hidden_bias_i,\
    input_weights_U_f, hidden_weights_W_f, hidden_bias_f,\
    input_weights_U_o, hidden_weights_W_o, hidden_bias_o,\
    input_weights_U_new, hidden_weights_W_new = param_toWeights(param)
    
    #Check what happens when we run the network using these weights?
    #Forward pass
    xs, hidden_states, outputs = {}, {}, {}
    input_gate, forget_gate, new_memory, final_memory = {}, {}, {}, {}
    loss = 0
    hidden_states[-1] = np.copy(hidden_state_prev)
    final_memory[-1] = np.copy(final_memory_prev)
    for t in range(0, S): 
        # stacking inputs into M x 1 vector
        xs[t] = np.zeros((input_size,1))  # 2 x 1
        xs[t][0] = adjClose[t] # the 1st element in the input is adjClose 
        xs[t][1] = tradeVolume[t] # the 2nd element in the input is tradeVolume 
        # Compute input gate K x 1
        input_gate[t] = sigmoid(input_weights_U_i@ xs[t] 
                                   + hidden_weights_W_i@ hidden_states[t-1] 
                                   + hidden_bias_i)
        # Compute forget gate K x 1
        forget_gate[t] = sigmoid(input_weights_U_f @ xs[t] 
                                   + hidden_weights_W_f @ hidden_states[t-1] 
                                   + hidden_bias_f)
        # Compute output gate N x 1
        outputs[t] = sigmoid(input_weights_U_o@ xs[t] 
                                   + hidden_weights_W_o@ hidden_states[t-1] 
                                   + hidden_bias_o)  
        # Generate new memory K x 1
        new_memory[t] = np.tanh(input_weights_U_new @ xs[t] 
                                   + hidden_weights_W_new @ hidden_states[t-1] )
        # Generate final memory K x 1 
        final_memory[t] = np.tanh(np.ravel(forget_gate[t])*np.ravel(final_memory[t-1]) 
                                   +  np.ravel(input_gate[t])*np.ravel(new_memory[t]))
        # Update hidden state K x 1
        hidden_states_array =  np.ravel(outputs[t])*final_memory[t]
        hidden_states[t] =  hidden_states_array.reshape(-1,1)
        # Compute root-mean-squared error as loss
        # - choose RMSE as loss, probablities are no longer in need.
        # - outputs[t] is numpy array, e.g. array([[0.5001818]]), indexing [0][0] is to take the number values.
        loss += (targets[t] -  outputs[t][0][0])**2
        
    loss_rmse = np.sqrt(loss/S)
    
    return loss_rmse

In [3]:
# LSTM STARTS:
#Prepare and download stock price 
start_date = datetime(2023,10,1)
end_date = datetime(2023,12,31)

stock = yf.download('AAPL',start_date ,end_date)
stock_price = pd.DataFrame(stock['Adj Close'])
stock_volume = pd.DataFrame(stock['Volume'])

scaler = MinMaxScaler()
price_scaled = pd.Series(scaler.fit_transform(stock_price).squeeze(),index=stock.index)
price_scaled.describe()
adjClose = price_scaled.values.tolist()
volume_scaled = pd.Series(scaler.fit_transform(stock_volume).squeeze(), index=stock.index)
volume_scaled.describe()
tradeVolume = volume_scaled.values.tolist()

#Initialize inputs and targets
input_size = 2  # M x 1
hidden_size = 3 # K x 1
output_size = 1 # N x 1
hidden_state_prev = np.zeros((hidden_size,1))  
final_memory_prev = np.zeros((hidden_size,1))  
targets = adjClose[1:] # we use data from 0 : T to predict adjusted close for 1 : T
S = len(targets)

#Initialize weights and biases (parameters)
# Input gate:
input_weights_U_i = np.random.randn(hidden_size, input_size) 
input_weights_U_i_stack = np.hstack(input_weights_U_i)
hidden_weights_W_i = np.random.randn(hidden_size, hidden_size) 
hidden_weights_W_i_stack = np.hstack(hidden_weights_W_i)
hidden_bias_i = np.zeros((hidden_size, 1)) 
hidden_bias_i_stack = np.hstack(hidden_bias_i)

# Forget Gate
input_weights_U_f = np.random.randn(hidden_size, input_size) 
input_weights_U_f_stack = np.hstack(input_weights_U_f)
hidden_weights_W_f  = np.random.randn(hidden_size, hidden_size) 
hidden_weights_W_f_stack = np.hstack(hidden_weights_W_f)
hidden_bias_f = np.zeros((hidden_size, 1)) 
hidden_bias_f_stack = np.hstack(hidden_bias_f)

# Output Gate
input_weights_U_o = np.random.randn(output_size, input_size) 
input_weights_U_o_stack = np.hstack(input_weights_U_o)
hidden_weights_W_o = np.random.randn(output_size, hidden_size) 
hidden_weights_W_o_stack = np.hstack(hidden_weights_W_o)
hidden_bias_o = np.zeros((output_size, 1)) 
hidden_bias_o_stack = np.hstack(hidden_bias_o)

# New Memory
input_weights_U_new = np.random.randn(hidden_size, input_size) 
input_weights_U_new_stack =  np.hstack(input_weights_U_new)
hidden_weights_W_new = np.random.randn(hidden_size, hidden_size) 
hidden_weights_W_new_stack = np.hstack(hidden_weights_W_new)

# ============================ # 
# PARAMETER ESTIMATION (TRAINING):
# ============================ #  
# Save timestamp
start = time.time()
param0 = np.hstack((input_weights_U_i_stack,hidden_weights_W_i_stack,hidden_bias_i_stack,
                    input_weights_U_f_stack,hidden_weights_W_f_stack,hidden_bias_f_stack,
                    input_weights_U_o_stack,hidden_weights_W_o_stack,hidden_bias_o_stack,
                    input_weights_U_new_stack, hidden_weights_W_new_stack))
results = minimize(LSTM_Loss, param0, method='BFGS', options={'disp': True})
param_star = results.x
# Save timestamp
end = time.time()
print(f"Time used for LSTM estimation: {end - start} seconds")
# ============================ #
# PARAMETER ESTIMATES: 
# ============================ #    
input_weights_U_i_star,\
hidden_weights_W_i_star,\
hidden_bias_i_star,\
input_weights_U_f_star,\
hidden_weights_W_f_star,\
hidden_bias_f_star,\
input_weights_U_o_star,\
hidden_weights_W_o_star,\
hidden_bias_o_star,\
input_weights_U_new_star,\
hidden_weights_W_new_star = param_toWeights(param_star)

print("Input Gate")
print("--------------------------------")
print('The optimal input weights U are:')
print(input_weights_U_i_star)
print('The optimal hidden state weights W are:')
print(hidden_weights_W_i_star)
print('The optimal hidden state bias b are:')
print(hidden_bias_i_star)
print("\n")

print("Forget Gate")
print("--------------------------------")
print('The optimal input weights U are:')
print(input_weights_U_f_star)
print('The optimal hidden state weights W are:')
print(hidden_weights_W_f_star)
print('The optimal hidden state bias b are:')
print(hidden_bias_f_star)
print("\n")

print("Output Gate")
print("--------------------------------")
print('The optimal output weights U are:')
print(input_weights_U_o_star)
print('The optimal hidden state weights W are:')
print(hidden_weights_W_o_star)
print('The optimal hidden state bias b are:')
print(hidden_bias_o_star)
print("\n")

print("New information")
print("--------------------------------")
print('The optimal output weights U are:')
print(input_weights_U_new_star)
print('The optimal hidden state weights W are:')
print(hidden_weights_W_new_star)
print("\n")

[*********************100%***********************]  1 of 1 completed


         Current function value: 0.039638
         Iterations: 239
         Function evaluations: 17876
         Gradient evaluations: 308
Time used for LSTM estimation: 45.832377672195435 seconds
Input Gate
--------------------------------
The optimal input weights U are:
[[-1.524397    1.3412312 ]
 [ 1.32681764  2.95004934]
 [-1.51343787 -1.16021097]]
The optimal hidden state weights W are:
[[ 1.36950544  2.14595873 -0.19001093]
 [-2.41500833 -2.09262971  1.30415715]
 [-1.70633441  0.87671303 -0.20254707]]
The optimal hidden state bias b are:
[[ 2.76363355]
 [ 2.2048926 ]
 [-0.6314254 ]]


Forget Gate
--------------------------------
The optimal input weights U are:
[[ 1.05910334  0.97328804]
 [-0.59272792 -1.24454029]
 [-0.74777394 -2.27097566]]
The optimal hidden state weights W are:
[[ 0.28305605 -0.30889531 -0.45880196]
 [-0.9200474  -2.5015827  -0.78458852]
 [ 1.88062721 -0.80808685 -0.3420175 ]]
The optimal hidden state bias b are:
[[ 2.24806681]
 [ 0.51626676]
 [-0.00507963]]


d:\Northeastern\CSYE-7380\venv\Lib\site-packages\scipy\optimize\_minimize.py:726: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)
